In [1]:
import pandas as pd
import geopandas as gpd
import glob

In [2]:
PATH = '/Users/dt/Documents/Projecten/northsea/data/'

## TODO

Create tables:

1. License: id, name, code, date, product, type, status, round, country, source
2. Operator: company_id, license_id, start_date, end_date, source
3. Licensees: company_id, license_id, share, start_date, end_date, source
4. Companies: id, name, address, country, registration, source
5. Com_to_com: child_id, parent_id, parent_name, address, country, share_parent, source
6. Field: id, license_id, name, source
7. Location: field_id, geometry
8. Production: field_id, year, commodity, production_1000_sm3
9. Prices: year, commodity, avg_price_usd, avg_price_eur
10. License_active_period: license_id, start_date, end_date
11. Authority: id, name, url, country, license_id

### License - NL

In [33]:
df_nl = pd.read_excel(PATH + 'license_and_production/nl_license_companies.xlsx')
len(df_nl)

768

In [34]:
# clean up column names

df_nl = df_nl.rename(columns={'product': 'commodity',
                              'status': 'status',
                              'license_name': 'name',
                              'license_code': 'code',
                              'license_status': 'status',
                              'license_type': 'type'
                            })

# add source and country

df_nl['source_url'] = 'https://www.nlog.nl/datacenter/prodfigures/licences'
df_nl['country'] = 'Netherlands'


In [35]:
df_nl.head(1)

,name,code,date,operator,commodity,type,status,source_url,country
0,'S-GRAVENZANDE,'S-GRAVENZANDE,NaT,NaN,Geothermal energy,Exploration licence,Rejected,https://www.nlog.nl/datacenter/prodfigures/lic...,Netherlands


In [36]:
#id, name, code, commodity, type, status, round, country, source

license_nl = df_nl[['name',
                    'code',
                    'commodity',
                    'type',
                    'status',
                    'country',
                    'source_url']]


### Operator - NL

In [ ]:
#Operator: company_id, license_id, start_date, end_date, source

In [94]:
df_nl = df_nl.dropna(subset=['operator'])

In [99]:
operator_nl = df_nl[['operator',
                     'code',
                     'date',
                     'source_url']]

### Licensees - NL

Licensees: company_id, license_id, share, start_date, end_date, source

In [100]:
# import shapefiles with licensees

nl_shape = gpd.read_file(PATH + 'shapes/nlog_licenses.shp')
len(nl_shape)

427

In [102]:
dedup = nl_shape[~nl_shape['LICENSEES'].isna()]

In [120]:
nl.head(1)

,LICENCE_DB,LICENCE_NM,LICENCE_TY,LICENSEES,OPERATOR,RESOURCE_N,ROUND,STATUS,SURFACE_AR,geometry
0,1.838341e+09,F05,Opsporingsvergunning,"EBN B.V., Nederlandse Aardolie Maatschappij B....",Neptune Energy Netherlands B.V.,"Koolwaterstoffen (Gas, Olie, Condensaat)",9,Onherroepelijk van kracht,398.0,"POLYGON ((585645.934 6077191.370, 607056.072 6..."


In [125]:
#Extract companies by exploding LICENCE_DB

nl = dedup.set_index('LICENCE_DB')\
    .LICENSEES.str.split(', ', expand=True)\
    .stack()\
    .reset_index()\
    .rename(columns={0:'name', 'LICENCE_DB': 'license_db'})

In [126]:
nl2 = pd.merge(nl, dedup, left_on='LICENSE_DB', right_on='license_db', how='left')

KeyError: 'license_db'

In [112]:
# Merge files to create file with shapes

licensee_nl = pd.merge(df_nl[['name', 'date']], 
                nl[['name']], 
                on='name',  
                how='inner')
len(licensee_nl)


0

In [13]:
# Merge again to create complete nl file

license_nl_licensees = pd.merge(dfnl[['license_db', 
                                      'licensee_name', 
                                      'LICENCE_NM',
                                      'geometry']], 
              df[['license_name',
                  'license_code',
                  'license_start_date',
                  'commodity',
                  'license_status',
                  'license_type']],
              left_on='LICENCE_NM', 
              right_on='license_name', 
              how='left')

len(license_nl_licensees)

864

In [14]:
# Clean it up and add country and source columns

license_nl_licensees = license_nl_licensees.drop('LICENCE_NM', 1)
license_nl_licensees['country'] = 'Netherlands'
license_nl_licensees['source_url'] = 'https://www.nlog.nl/datacenter/prodfigures/licences'
license_nl_licensees = license_nl_licensees.drop_duplicates()
len(license_nl_licensees)

/var/folders/7q/y68bnjks05d1j1jyn8myxvqw0000gn/T/ipykernel_8603/1589768911.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  license_nl_licensees = license_nl_licensees.drop('LICENCE_NM', 1)


864

In [ ]:
license_nl_licensees.to_csv('/Users/dt/Downloads/nl_licensees.csv', index=False)

In [ ]:


# create licensee df

license_no_licensees = df[['license_name',
                         'license_code',
                         'license_start_date',
                         'license_end_date',
                         'licensee_name',
                         'licensee_interest',
                         'licensee_registration_number',
                         'country',
                         'source_url']]


len(license_no_licensees)

In [ ]:
#read operator

df = pd.read_csv(PATH + 'license_and_production/no_license_operator_hist.csv')
len(df)

In [ ]:
# transform date columns

df['prlOperDateValidFrom'] = pd.to_datetime(df['prlOperDateValidFrom']).dt.strftime('%Y-%m-%d')
df['prlOperDateValidTo'] = pd.to_datetime(df['prlOperDateValidTo']).dt.strftime('%Y-%m-%d')

In [ ]:
# rename columns

df = df.rename(columns={'prlName': 'license_name',
                        'prlOperDateValidFrom': 'license_start_date',
                        'prlOperDateValidTo': 'license_end_date',
                        'prlNpdidLicence': 'license_code',
                        'cmpLongName': 'operator_name'
                        })

# create country and source columns

df['source_url'] = 'https://factpages.npd.no/en/licence'
df['country'] = 'Norway'

# create operator df

license_no_operators = df[['license_name',
                             'license_code',
                             'license_start_date',
                             'license_end_date',
                             'operator_name',
                           'source_url',
                           'country']]

len(license_no_operators)

## License - NO

In [37]:
#read licensees

df_no = pd.read_csv(PATH + 'license_and_production/no_licence.csv')
len(df)

28903

In [38]:
df_no.columns

Index(['prlName', 'prlLicenseeDateValidFrom', 'prlLicenseeDateValidTo',
       'cmpLongName', 'prlLicenseeInterest', 'prlLicenseeSdfi',
       'prlOperDateValidFrom', 'prlOperDateValidTo', 'prlNpdidLicence',
       'cmpNpdidCompany', 'prlLicenseeDateUpdated', 'DatesyncNPD'],
      dtype='object')

In [40]:
df_no.sample(5)

,prlName,prlLicenseeDateValidFrom,prlLicenseeDateValidTo,cmpLongName,prlLicenseeInterest,prlLicenseeSdfi,prlOperDateValidFrom,prlOperDateValidTo,prlNpdidLicence,cmpNpdidCompany,prlLicenseeDateUpdated,DatesyncNPD
9182,110,01.03.1985,14.02.1993,Norsk Hydro Produksjon AS,10.0,NaN,NaN,NaN,21684,430,10.03.2022,29.03.2022
16580,286,02.01.2007,30.09.2007,Eni Norge AS,20.0,NaN,NaN,NaN,2073578,2388594,10.03.2022,29.03.2022
15740,262,18.05.2016,29.11.2016,PGNiG Upstream International AS,15.0,NaN,NaN,NaN,1033093,23412029,10.03.2022,29.03.2022
21036,496,23.01.2009,08.03.2011,Premier Oil Norge AS,70.0,NaN,23.01.2009,08.03.2011,5106035,3482067,10.03.2022,29.03.2022
4868,055 B,10.09.2003,30.12.2003,Norsk Agip AS,13.2,NaN,NaN,NaN,102499,45,10.03.2022,29.03.2022


In [20]:
# transform dates

df_no['prlLicenseeDateValidFrom'] = pd.to_datetime(df_no['prlLicenseeDateValidFrom']).dt.strftime('%Y-%m-%d')
df_no['prlLicenseeDateValidTo'] = pd.to_datetime(df_no['prlLicenseeDateValidTo']).dt.strftime('%Y-%m-%d')

In [48]:
#Rename columns

df_no = df_no.rename(columns={'prlName': 'name',
                              'prlNpdidLicence': 'code',
                              })

# add country and source columns

df_no['country'] = 'Norway'
df_no['source_url'] = 'https://factpages.npd.no/en/licence'

In [43]:
df_no2 = pd.read_excel(PATH + 'license_and_production/no_fields.xlsx')

In [54]:
df_no2 = df_no2.rename(columns={'Owner name': 'name',
                        'Owner kind': 'type',
                        'NPDID owner': 'code',
                        'Current activity status': 'status'})

In [82]:
license_no = pd.merge(df_no[['name', 'code', 'country', 'source_url']],
                      df_no2[['code', 'type', 'status']],
                      on = 'code',
                      how = 'right'
                     
                     )

In [83]:
len(license_no.dropna(subset=['type']))

5272

In [92]:
license_no[license_no.code == 20900.0]

Shut down    772
Producing    579
Name: status, dtype: int64

In [68]:
license_no.status.value_counts()

Producing                  3591
Shut down                  1247
Approved for production     390
Name: status, dtype: int64

In [87]:
df = pd.read_excel(PATH + 'license_and_production/no_field_description.xlsx')

In [88]:
df.head()

,Field name,Type,Text,NPDID field,Date updated,Date sync NPD
0,ALBUSKJELL,Development,Albuskjell is a field in the southern part of ...,43437,2020-02-08,12.04.2022
1,ALBUSKJELL,Reservoir,Albuskjell produced gas and condensate from Ma...,43437,2018-03-16,12.04.2022
2,ALBUSKJELL,Recovery,The field was produced by pressure depletion.,43437,2017-04-11,12.04.2022
3,ALBUSKJELL,Transport,The well stream was transported via pipeline t...,43437,2018-03-16,12.04.2022
4,ALBUSKJELL,Status,The field was shut down in 1998 and the platfo...,43437,2021-02-02,12.04.2022


In [90]:
df['Field name'].nunique()

128

## United Kingdom

In [ ]:
# import file with current license information

df = pd.read_excel(PATH + 'license_and_production/uk_license_companies.xlsx')
len(df)

In [ ]:
#rename columns

df = df.rename(columns={'group_name': 'licensee_parent',
                      'company': 'licensee_name',
                      'Block / Subarea': 'asset',
                      'Interest': 'licensee_interest',
                      'Operator': 'operator_name',
                      'Licence': 'license_name'})

# create country and source columns

df['country'] = 'United Kingdom'
df['source_url'] = 'https://www.nstauthority.co.uk/data-centre/data-downloads-and-publications/licence-data/'

# clean up interest

df['licensee_interest'] = df['licensee_interest'].str.replace('%', '', regex=False)
len(df)

In [ ]:
# create current licensee df

license_uk_licensees_current = df[['licensee_name',
                           'asset',
                           'licensee_interest',
                           'license_name',
                           'country',
                           'source_url']]

# create df with parent companies

license_uk_parent_current = df[['licensee_parent',
                        'licensee_name']]

# create df with current operators

license_uk_operators_current = df[['operator_name',
                                   'license_name',
                                   'country',

                                   'source_url']]

In [ ]:
# import historical license data

df = pd.read_excel(PATH + 'license_and_production/uk_2014-2020-field-equity-shares-june-2021.xlsx', skiprows=2)
len(df)

In [ ]:
# rename columns

df = df.rename(columns={'Organisation Name': 'licensee_name',
                      'Percentage Holding': 'licensee_interest',
                      'Start Date': 'license_start_date',
                      'End Date': 'license_end_date',
                      'Field Name': 'license_name'})

# create country and source columns

df['country'] = 'United Kingdom'
df['source_url'] = 'https://www.nstauthority.co.uk/data-centre/data-downloads-and-publications/licence-data/'

len(df)

In [ ]:
# create historical licensee df

license_uk_licensees_hist = df[['licensee_name', 
                                   'license_name',
                                   'license_start_date',
                                   'license_end_date',
                                   'country',
                                   'source_url']]

In [ ]:
# filter out operator info from df and change column name

df_operator = df[df['Operator Flag'] == 'Y']
df_operator = df_operator.rename(columns={'licensee_name': 'operator_name'})

# create historical operator df

license_uk_operators_hist = df_operator[['operator_name',
                                         'license_start_date',
                                         'license_end_date',
                                         'country',
                                         'source_url']]



In [ ]:
# concat licensee dfs - uk might contain duplicates

licensees = pd.concat([license_uk_licensees_current, 
                       license_uk_licensees_hist, 
                       license_no_licensees, 
                       license_nl_licensees])

In [ ]:
# concat operators dfs - uk might contain duplicates

operators = pd.concat([license_uk_operators_hist, 
                       license_uk_operators_current,
                       license_no_operators, 
                       license_nl_operators])

In [ ]:
# concat parent dfs

parents = pd.concat([license_uk_parent_current])

### Clean companies - licensees and operators

In [ ]:
licensees.licensee_name.value_counts()[40:60]

In [ ]:
licensees[licensees['licensee_name'].str.contains('Gazpr', na=False)]

## Production - Netherlands

In [ ]:
def parse_commodity(commodity):
    dfs = []
    for file in glob.glob(PATH + 'license_and_production/nl_' + commodity + '_produced_*.xlsx'):
        df = pd.read_excel(file, skiprows=1, skipfooter=1)
        df['total_1000_sm3'] = df.iloc[:, -12:].sum(axis=1)
        if commodity == 'oil':
            df['total_1000_sm3'] = df['total_1000_sm3'] / 1000
        df['commodity'] = commodity
        df = df.dropna(subset=['YEAR'])
        df.YEAR = df.YEAR.astype('int')
        df = df.rename(columns={'LICENCE': 'license_name',
                                'FIELD': 'license_name',
                            'OPERATOR': 'operator_name',
                            'YEAR': 'production_year'
                           })

        nl_production = df[['license_name', 
                        'operator_name',
                        'commodity',
                        'production_year',
                        'total_1000_sm3'
                        ]]
        dfs.append(nl_production)

    df = pd.concat(dfs)
    return df


In [ ]:
df = pd.concat([parse_commodity('gas'), parse_commodity('oil')])

In [ ]:
df

## Production - Norway

In [ ]:
df_field_licensee = pd.read_excel(PATH + 'license_and_production/no_field_licensee_hst.xlsx')
len(df_field_licensee)

In [ ]:
df_field_licensee.head()

In [ ]:
df_field_licensee['Owner name'].value_counts()

In [ ]:
df_field_operator = pd.read_excel(PATH + 'license_and_production/no_field_operator_hst.xlsx')
len(df_field_operator)

In [ ]:
df_field_operator.head()

In [ ]:
df_field_owner = pd.read_excel(PATH + 'license_and_production/no_field_owner_hst.xlsx')
len(df_field_owner)

In [ ]:
df_field_owner.head()

In [ ]:
df_field_owner[df_field_owner['NPDID owner'] == ]

In [ ]:
df_field_owner[df_field_owner['Owner name'].str.contains('STATFJORD ØST UNIT')]

In [ ]:
df_field_production = pd.read_excel(PATH + 'license_and_production/no_field_production_yearly.xlsx')
len(df_field_production)

In [ ]:
df_field_production.tail()

In [ ]:
df_field_production = df_field_production\
                        .rename(columns={'Field (Discovery)': 'license_name',
                                         'Year': 'production_year',
                                         'Net - oil \n[mill Sm3]': 'total_oil_1000_sm3',
                                         'Net - gas \n[bill Sm3]': 'total_gas_1000_sm3'})
df_field_production['total_oil_1000_sm3'] = df_field_production['total_oil_1000_sm3'] * 1000
df_field_production['total_gas_1000_sm3'] = df_field_production['total_gas_1000_sm3'] * 1000000

df_oil = df_field_production[['license_name',
                              'production_year',
                              'total_oil_1000_sm3']]

df_oil['commodity'] = 'oil'
df_oil = df_oil.rename(columns={'total_oil_1000_sm3': 'total_1000_sm3'})

df_gas = df_field_production[['license_name',
                              'production_year',
                              'total_gas_1000_sm3']]

df_gas['commodity'] = 'gas'
df_gas = df_gas.rename(columns={'total_gas_1000_sm3': 'total_1000_sm3'})

df = pd.concat([df_oil, df_gas])



In [ ]:
df.tail()